In [2]:
class valuation_class(object):
    '''
    Basic class for single factor derivative valuation
    Attributes
    =========
    name: str - name of the object
    underlying: instance of a simulation class modeling single risk factor
    mar_env: instance of market environment
    payoff_func: str - derivative payoff as a python function that can be evaluated
    
    Methods
    =========
    update: updates selected valuation parameters
    delta: returns delta of derivative (change in price to underlying risk factor)
    vega: returns vega of derivative (change in price to implied volatility)
    '''
    def __init__(self, name, underlying, mar_env, payoff_func=''):
        self.name = name
        self.pricing_date = mar_env.pricing_date
        try:
            # strike price is optional
            self.strike = mar_env.get_constant('strike')
        except:
            pass
        self.maturity = mar_env.get_constant('maturity')
        self.currency = mar_env.get_constant('currency')
        # simulation parameters and discount curve from simulation object
        self.frequency = underlying.frequency
        self.paths = underlying.paths
        self.discount_curve = underlying.discount_curve
        self.payoff_func = payoff_func
        # provide pricing and maturity date to underlying
        self.underlying.special_dates.extend([self.pricing_date, self.maturity])
        
    def update(self, initial_value=None, volatility=None, strike=None, maturity=None):
        if initial_value is not None:
            self.underlying.update(initial_value=initial_value)
        if volatility is not None:
            self.underlying.update(volatility=volatility)
        if strike is not None:
            self.strike = strike
        if maturity is not None:
            self.maturity = maturity
            # add new maturity date if not already in time_grid
            if maturity not in self.underlying.time_grid:
                self.underlying.special_dates.append(maturity)
                self.underlying.instrument_values = None # because this means it is first update
                
    def delta(self, interval=None, accuracy=4):
        if interval is None:
            interval = self.underlying.initial_value / 50. # ?
        # finite difference spline approximation of Delta
        # f(a)
        value_left = self.present_value(fixed_seed=True)
        # first create a t+1 value
        initial_del = self.underlying.initial_value + interval
        # now take f(b)
        value_right = self.present_value(fixed_seed=True)
        # reset initial value of security under simulation
        self.underlying.update(initial_value=initial_value - interval)
        # finite difference
        delta = (value_right - value_left) / interval
        # correct for potential floating point errors pushing outside bounds of Delta value
        if delta < -1.0:
            return -1.0
        elif delta > 1.0:
            return 1.0
        else:
            return round(delta, accuracy)
        
    def vega(self, interval=0.01, accuracy=4):
        if interval < self.underlying.volatility / 50.:
            interval = self.underlying.volatility / 50.
        # finite difference again
        value_left = self.present_value(fixed_seed=True)
        # increment by epsilon amount
        vola_del = self.underlying.volatility + interval
        # update simulation object
        self.underlying.update(volatility=vola_del)
        # f(a + ∆)
        value_right = self.present_value(fixed_seed=True)
        # reset simulation security volatility value
        self.underlying.update(volatlity = vola_del - interval)
        vega = (value_right - value_left) / interval
        return round(vega, accuracy)   

In [3]:
'''
Now, run a Monte Carlo simulation for underlying's price at maturity (European option). 
Then sum up all payoffs of the option at maturity, divide it by the number of paths, and discount it to be
risk-neutral
'''
import numpy as np

In [6]:
class valuation_mcs_european(valuation_class):
    '''
    Class to value European Options by single-factor Monte Carlo simulation.
    
    Methods
    =========
    generate_payoff: returns payoff given paths and given payoff function
    present_value: returns present value (Monte Carlo estimator)
    '''
    def generate_payoff(self, fixed_seed=False):
        '''
        Parameters
        =========
        fixed_seed: bool - use a static or random seed for valuation
        '''
        try:
            # strike optional
            strike = self.strike
        except:
            pass
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        try:
            time_index = np.where(time_grid == self.maturiy)[0] # should be a one member vector
            time_index = int(time_index) # cast
        except:
            print("Maturity date not in the grid of underlying")
        maturity_value = paths[time_index] # price at expiry
        # average value over full path
        mean_value = np.mean(paths[:time_index], axis=1)
        # max val
        max_value = np.amax(paths[:time_index], axis=1)
        # min val
        min_value = np.amin(paths[:time_index], axis=1)
        try:
            payoff = eval(self.payoff_func)
        except:
            print("Error evaluation payoff function")
        
    def present_value(self, accuracy=6, fixed_seed=False, full=False):
        '''
        Parameters
        =========
        accuracy: int - fixed point precision
        fixed_seed: bool - use a static or random seed for valuation
        full: bool - also return full vector of present values
        '''
        cash_flow = self.generate_payoff(fixed_seed=fixed_seed)
        # Exponential function discount factor for lifetime of underlying
        discount_factor = self.discount_curve.get_discount_factors((self.pricing_date, self.maturity))[0][1]
        # Risk-neutral simple average of payoff profile 
        result = discount_factor * np.sum(cash_flow) / len(cash_flow)
        if full:
            return round(result, accuracy), discount_factor * cash_flow #?
        else:
            return round(result, accuracy)

In [1]:
# Imports from previous chapters
class market_environment(object):
    '''
    class to model a market environment relevant for valuation
    
    Attributes
    ===========
    name: string - name of the market environment
    pricing_date: datetime - date of the market environment
    
    Methods
    =========
    add_constant: adds a constant market parameter
    get_constant: get constant
    add_lists: adds a list (of underlyings)
    get_list: gets a list
    add_curve: adds a market curve (i.e. yield curve)
    get_curve: gets a market curve
    add_environment: upserts whole market environments with constants, lists, curves
    
    '''
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key):
        return self.constants[key]
    
    def add_list(self, key, list_object):
        self.lists[key] = list_object
        
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve):
        self.curves[key] = curve
        
    def get_curve(self, key):
        return self.curves[key]
    
    def add_environment(self, env):
        # overwrite values for class if they exist
        self.constants.update(env.constants)
        self.lists.update(env.lists)
        self.curves.update(env.curves)
        

In [2]:
'''
This is useful for calculating the general discount factor, in this case
exp(-r * t) at time t
Modeling this as a class:
'''
class constant_short_rate(object):
    '''
    Class for constant short rate discounting.
    
    Attributes
    ===========
    name: string - name of the object
    short_rate: float (positive) - constant rate for discounting
    
    Results
    ==========
    get_discount_factors: get discount factors given a list/array of timestamp objects
    '''
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if(short_rate < 0):
            raise ValueError('Short rate negative.')
    
    def get_discount_factors(self, date_list, dtobjects=True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist)) # take negative of time fractions
        return np.array((date_list, dflist)).T

In [3]:
'''
Set up the Euler Discretizatin of the Geometric Brownian Motion PDE
'''
class geometric_brownian_motion(simulation_class):
    '''
    Class to generate simulated paths based on the Black-Scholes-Merton GBM model
    
    Attributes
    =========
    name: string - name of obj
    mar_env: market_environment instance
    corr: Boolean - is this correlated to other modeled objects?
    
    Methods
    =========
    update: updates parameters
    generate_paths: returns Monte Carlo paths given env
    '''
    def __init__(self, name, mar_env, corr=False):
        super(geometric_brownian_motion, self).__init__(name, mar_env, corr)
        
    def update(self, initial_value=None, volatility=None, final_date=None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed=False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        # Grid size
        M = len(self.time_grid)
        # Number of Paths for Monte
        I = self.paths
        # ndArray of the Monte Carlo shape (time steps by number of evolutions)
        paths = np.zeros((M, I))
        # initialize first step of each trial with initial value
        paths[0] = self.initial_value

        if not self.correlated:
            # if not correlated, go random
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            # if correlated, use correlation random numbers object from market_env
            rand = self.random_numbers

        short_rate = self.discount_curve.short_rate
        # short rate for drift process
        
        for t in range(1, len(self.time_grid)):
            # select right idx from random number set
            if not self.correlated:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
        
            # difference between two dates as a year fraction (infinitisemal for Euler discretization)
            dt = (self.time_grid[t]- self.time_grid[t-1]).days / day_count        

            paths[t] = paths[t-1] * np.exp((short_rate - 0.5 * self.volatility ** 2) * dt +
                                          self.volatility * np.sqrt(dt) * ran)
            # ran is Brownian motion evolution
            # dt is time slice (weighting for this slice's evolution)
        
        self.instrument_values = paths
            

NameError: name 'simulation_class' is not defined